Team Members

*   Libin Kutty - Group 2
*   Ritu Gahir - Group 3
*   Viju Sudhi - Group 10

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
print(os.getcwd())
os.chdir("/content/drive/My Drive/Colab Notebook/IDL/")
print(os.getcwd())

/content
/content/drive/My Drive/Colab Notebook/IDL


In [140]:
!python prepare_data2.py shakespeare_input.txt skpidl6 \\n\\n+ --maxlen 500

2020-06-02 09:18:23.569411: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
29429 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
29429 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [6]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skpidl6.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))
data = data.padded_batch(128, 500, drop_remainder=True)

# a map from characters to indices
vocab = pickle.load(open("skpidl6_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'j': 3, 'B': 4, 'o': 5, 'C': 6, 'n': 7, 'U': 8, 'V': 9, 'm': 10, 'z': 11, 'i': 12, ' ': 13, 'x': 14, 'k': 15, '?': 16, 'R': 17, 'f': 18, ';': 19, 'd': 20, 'v': 21, 'g': 22, 'Q': 23, 'e': 24, ',': 25, '-': 26, 'q': 27, '.': 28, 'D': 29, 'K': 30, 'h': 31, 'l': 32, 'F': 33, 'T': 34, 'O': 35, ':': 36, 'a': 37, 'P': 38, 'p': 39, 'r': 40, 'c': 41, '&': 42, 'E': 43, ']': 44, '3': 45, 'w': 46, 'I': 47, 'Y': 48, 'L': 49, '\n': 50, 'M': 51, 'W': 52, "'": 53, 'A': 54, 'b': 55, 'G': 56, 'N': 57, 'X': 58, '[': 59, 's': 60, 'H': 61, 'J': 62, 'Z': 63, 't': 64, '!': 65, 'S': 66, 'u': 67, '$': 68, 'y': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


In [0]:
@tf.function
def loss_fn(labels, logits):
  return tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)

In [0]:
class RNNModel():
  def __init__(self, vocab_size, batch_size):
    self.batch_size = batch_size
    self.vocab_size = vocab_size
#    self.loss_func = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
#    self.loss_func = tf.nn.sparse_softmax_cross_entropy_with_logits()
    self.opt = tf.optimizers.Adam()
    self.loss = 0.0

  def build(self, embedding_dim, units):
    self.model = tf.keras.Sequential()
    self.model.add(tf.keras.layers.Embedding(self.vocab_size, embedding_dim, batch_input_shape=[self.batch_size,70]))
    self.model.add(tf.keras.layers.SimpleRNN(units, return_sequences=True, stateful=True))
    self.model.add(tf.keras.layers.Dense(self.vocab_size))

    return self.model

  #@tf.function
  def train(self, seq_batch):
    loss_ts = tf.TensorArray(tf.float32, size=128, dynamic_size=True)
    loss_temp = []
    ts = 0
    mask = []

    with tf.GradientTape() as t:

      for i in tf.range(tf.shape(seq_batch)[1] - 1):
        mask = []
        inp_curr = seq_batch[:,i]
        inp_next = seq_batch[:, i+1]

        inp_curr = tf.one_hot(inp_curr,self.vocab_size)   
        inp_next = tf.cast(tf.one_hot(inp_next,self.vocab_size), tf.int64) 

        count_non_zero = tf.math.count_nonzero(seq_batch[:,i]) - 1
        mask_seq = tf.sequence_mask([count_non_zero], maxlen=500, dtype=tf.float32)
        mask.append(mask_seq)

        output = self.model(inp_curr)

#        self.loss = self.loss_func(seq_batch[:, i+1], output)
        self.loss = loss_fn(inp_next, output)
        self.loss = tf.math.multiply(self.loss,mask_seq)

        loss_ts = loss_ts.write(ts, self.loss)
        ts +=1
        #loss_ts.append(loss_temp)

  
      loss_ts = loss_ts.stack()
      # self.loss = tf.reduce_mean(loss_ts)
      self.loss = tf.reduce_sum(loss_ts)
      self.loss = self.loss / tf.cast(count_non_zero, dtype=tf.float32)

    #loss_temp = loss_temp/count


    varis = self.model.trainable_variables
    
    grads = t.gradient(self.loss, varis)
#    grads, _ = tf.clip_by_global_norm(grads,1)
    self.opt.apply_gradients(zip(grads, varis))

    self.model.reset_states()
    return self.loss


In [0]:
rnn = RNNModel(vocab_size, 128)

In [36]:
rnn.build(500, 256)

In [49]:
mask = []
count = 0
for step, seqs in enumerate(data):
  print(tf.shape(seqs))
  inp = seqs[:,1]
  inp_curr = tf.one_hot(seqs[:,1], vocab_size)
  print(inp_curr.shape)
  print(inp_curr[])
  for i in range(70):
    count_non_zero = tf.math.count_nonzero(inp[:])
    count +=count_non_zero
    mask_seq = tf.sequence_mask([count_non_zero], maxlen=500, dtype=tf.float32)
    mask_batch = tf.convert_to_tensor(mask_seq, tf.float32)
    mask.append(mask_batch)
    print(mask_batch.shape)
    break

  break  

tf.Tensor([128 500], shape=(2,), dtype=int32)
(128, 70)
tf.Tensor([], shape=(0, 70), dtype=float32)
(1, 500)


In [37]:

for step, seqs in enumerate(data):
    loss_avg = rnn.train(seqs)

    if not step % 10:
        print("Step: {} Loss: {}".format(step, loss_avg))
        print()

    if step > steps:
      break

InvalidArgumentError: ignored

In [0]:
def sample(length, model):

  #x = np.zeros((1, vocab_size), dtype=np.float32)
  x = [0]
  
  ixes = []
  for i in range(length):
    inp_curr = tf.one_hot(x[-1:],depth=model.vocab_size)
    inp_curr = tf.reshape(inp_curr, [1, -1, 70])
    y = rnn.model(inp_curr)
    output = tf.nn.softmax(y)

    ix = np.random.choice(range(vocab_size), p=np.ravel(output))
    #x = np.zeros((1, vocab_size), dtype=np.float32)
    #x[ix] = 1
    x.append(ix)
    
    ixes.append(ix)
    if ix == 2:
      break

  return ixes

In [228]:
sample_ix = sample(500, rnn)
txt = ''.join(ind_to_ch[ix] for ix in sample_ix)
print(txt)

ValueError: ignored